In [32]:
import numpy as np
import sys
sys.path.insert(0,'..')
import xbrl.envs as bandits
from xbrl.algs.nnmodel import MLLinearNetwork
from xbrl.algs.knownrep.leaderselect import LeaderSelectElim
import xbrl.envs.hlsutils as hlsutils
import torch
import torch.nn as nn
import math

In [33]:
ncontexts = 5000
narms = 10
dim = 20
contextgeneration= "gaussian"
feature_expansion = "none"
seed_problem=99

In [34]:
features, theta = bandits.make_synthetic_features(
    n_contexts=ncontexts, n_actions=narms, dim=dim,
    context_generation=contextgeneration, feature_expansion=feature_expansion,
    seed=seed_problem
)
print(features.shape)

(5000, 10, 20)


In [35]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels + m.in_channels
            m.weight.data.normal_(0, math.sqrt(4. / n))
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            n = m.in_features + m.out_features
            m.weight.data.normal_(0, math.sqrt(100. / n))
            if m.bias is not None:
                m.bias.data.zero_()

In [36]:
hid_dim = [60, 10]
layers = [(el, nn.Tanh()) for el in hid_dim]
net = MLLinearNetwork(dim, layers)
initialize_weights(net)

In [37]:
U = features.reshape(-1, dim)
print(features.shape, U.shape)
xt = torch.tensor(U, dtype=torch.float)
H = net.embedding(xt).cpu().detach().numpy()
print(H.shape)
newfeatures = H.reshape(ncontexts, narms, net.embedding_dim)
R = net(xt).detach().numpy().ravel()
newreward = R.reshape(ncontexts, narms)

theta = net.fc2.weight.detach().numpy().ravel()
y = newfeatures @ theta
assert np.allclose(y, newreward)

(5000, 10, 20) (50000, 20)
(50000, 10)


In [38]:
print(f"New rep -> HLS rank: {hlsutils.hls_rank(newfeatures, newreward)} / {newfeatures.shape[2]}")
print(f"New rep -> is HLS: {hlsutils.is_hls(newfeatures, newreward)}")
print(f"New rep -> HLS min eig: {hlsutils.hls_lambda(newfeatures, newreward)}")
print(f"New rep -> is CMB: {hlsutils.is_cmb(newfeatures, newreward)}")
print(f"features norm: {np.linalg.norm(newfeatures, axis=-1).max()}")
print(f"reward range: {R.min()}, {R.max()}")

New rep -> HLS rank: 10 / 10
New rep -> is HLS: True
New rep -> HLS min eig: 0.21417641653139144
New rep -> is CMB: True
features norm: 3.1622774600982666
reward range: -23.99169921875, 24.056562423706055


In [57]:
NEW_LAYERS = [
    layers, #as the ground truth
    [(300, nn.Tanh())],
    [(10, nn.Tanh()), (10, nn.Tanh())]
]

misspecified_nets = []
for L in NEW_LAYERS:
    net2 = MLLinearNetwork(dim, L)
    initialize_weights(net2)
    print(net2)

    H = net2.embedding(xt).cpu().detach().numpy()
    H = H.reshape(ncontexts, narms, net2.embedding_dim)
    R = net2(xt).detach().numpy().ravel()
    R = R.reshape(ncontexts, narms)
    errors = np.abs(newreward - R)
    print(f"max error: {errors.max()}")
    print(f"min error: {errors.min()}")
    print(f"avg error: {errors.mean()}")
    print(f"New rep -> is HLS: {hlsutils.is_hls(H, R)}")
    print(f"New rep -> HLS min eig: {hlsutils.hls_lambda(H, R)}")
    print("\n\n")
    misspecified_nets.append(net2)

MLLinearNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=10, bias=True)
    (3): Tanh()
  )
  (fc2): Linear(in_features=10, out_features=1, bias=False)
)
max error: 42.27394104003906
min error: 0.0003643035888671875
avg error: 8.838642120361328
New rep -> is HLS: True
New rep -> HLS min eig: 0.3729509800119442



MLLinearNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=300, bias=True)
    (1): Tanh()
  )
  (fc2): Linear(in_features=300, out_features=1, bias=False)
)
max error: 49.63204574584961
min error: 0.00025784969329833984
avg error: 11.575180053710938
New rep -> is HLS: True
New rep -> HLS min eig: 0.039743458476127855



MLLinearNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Tanh()
  )
  (fc2): Linear(in_features=10, o

In [58]:
# save info
import os
dir_path = "../data/misspecified_example1"

try:
    os.mkdir(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

txtf = open(os.path.join(dir_path, "README.txt"), "w")
txtf.write("Generated using generate_nn_hls_problems.ipynb")
txtf.write(f"Problem Properties\nnum contexts: {features.shape[0]}\nnum actions: {features.shape[1]}\ndim: {features.shape[2]}\n")

for i, m in enumerate([net] + misspecified_nets):
    torch.save(m, os.path.join(dir_path, f"net{i}.pth"))
    text = m.__repr__()
    txtf.write("="*30 + "\n")
    txtf.write(f"net{i}.pth\n")
    txtf.write("-"*30 + "\n")
    txtf.write(text)
    txtf.write("\n")
    H = m.embedding(xt).cpu().detach().numpy()
    H = H.reshape(ncontexts, narms, m.embedding_dim)
    R = m(xt).detach().numpy().ravel()
    R = R.reshape(ncontexts, narms)
    txtf.write(f"Following properties are computed wrt the network predictions\n")
    txtf.write(f"HLS rank: {hlsutils.hls_rank(H, R)} / {H.shape[2]}\n")
    txtf.write(f"is HLS: {hlsutils.is_hls(H, R)}\n")
    txtf.write(f"HLS min eig: {hlsutils.hls_lambda(H, R)}\n")
    txtf.write(f"is CMB: {hlsutils.is_cmb(H, R)}\n")
    txtf.write("\n")


txtf.close()

with open(os.path.join(dir_path, 'features.npy'), 'wb') as f:
    np.save(f, features)

Error: ../data/misspecified_example1 : File exists
